In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] ='false'
# os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'

# Disable GPU
import tensorflow as tf

tf.config.set_visible_devices([], "GPU")
# os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Use GPU 0 for PyTorch

2023-08-21 21:07:58.233393: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 21:07:58.714329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-21 21:07:59.328274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 21:07:59.364060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import umap
import cuml
import pandas as pd
import datashader as ds
import datashader.transfer_functions as ds_tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
import datashader.utils as ds_utils
from umap.parametric_umap import ParametricUMAP
import numpy as np

/home/ilker/miniconda3/envs/thesis-work/lib/python3.10/site-packages/cudf/utils/gpu_utils.py:148: UserWarning: No NVIDIA GPU detected
  warnings.warn("No NVIDIA GPU detected")


CudaSupportError: Error at driver init: Call to cuInit results in CUDA_ERROR_NO_DEVICE (100)

In [3]:
pal = [
    "#9e0142",
    "#d8434e",
    "#f67a49",
    "#fdbf6f",
    "#feeda1",
    "#f1f9a9",
    "#bfe5a0",
    "#74c7a5",
    "#378ebb",
    "#5e4fa2",
]
color_key = {str(d): c for d, c in enumerate(pal)}

## UMAP

- [bert-loves-chemisrty example](https://github.com/seyonechithrananda/bert-loves-chemistry/blob/master/chemberta/visualization/ChemBERTA_dimensionaliy_reduction_BBBP.ipynb)

### TIMES:

- Kinase (68k): 3 min 20s
  - n_neighbors=50, n_components=2


In [ ]:
reducer = umap.UMAP(
    n_neighbors=15, n_components=2, n_epochs=500, min_dist=0.1, random_state=42
)
reducer = cuml.UMAP(n_neighbors=15, n_components=2, n_epochs=500, min_dist=0.1)

embedding = reducer.fit_transform(data)

df = pd.DataFrame(embedding, columns=("x", "y"))
df["class"] = pd.Series([str(x) for x in target], dtype="category")

cvs = ds.Canvas(plot_width=400, plot_height=400)
agg = cvs.points(df, "x", "y", ds.count_cat("class"))
img = ds_tf.shade(agg, color_key=color_key, how="eq_hist")

ds_utils.export_image(img, filename="fashion-mnist", background="white")

image = plt.imread("fashion-mnist.png")
fig, ax = plt.subplots(figsize=(12, 12))
plt.imshow(image)
plt.setp(ax, xticks=[], yticks=[])
plt.title(
    "Fashion MNIST data embedded\n"
    "into two dimensions by UMAP\n"
    "visualised with Datashader",
    fontsize=12,
)

plt.show()

## PARAMETRIC UMAP


### Dataset

In [ ]:
(train_images, Y_train), (test_images, Y_test) = mnist.load_data()
train_images = train_images.reshape((train_images.shape[0], -1)) / 255.0
test_images = test_images.reshape((test_images.shape[0], -1)) / 255.0

### Encoder-Decoder Network

In [4]:
dims = (28, 28, 1)
n_components = 2

In [5]:
encoder = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=dims),
        tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
        ),
        tf.keras.layers.Conv2D(
            filters=128,
            kernel_size=3,
            strides=(2, 2),
            activation="relu",
            padding="same",
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512, activation="relu"),
        tf.keras.layers.Dense(units=512, activation="relu"),
        tf.keras.layers.Dense(units=n_components),
    ]
)
# encoder.summary()

In [6]:
decoder = tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(n_components)),
        tf.keras.layers.Dense(units=512, activation="relu"),
        tf.keras.layers.Dense(units=7 * 7 * 256, activation="relu"),
        tf.keras.layers.Reshape(target_shape=(7, 7, 256)),
        tf.keras.layers.Conv2DTranspose(
            filters=128,
            kernel_size=3,
            strides=(2, 2),
            padding="SAME",
            activation="relu",
        ),
        tf.keras.layers.Conv2DTranspose(
            filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation="relu"
        ),
        tf.keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=3,
            strides=(1, 1),
            padding="SAME",
            activation="sigmoid",
        ),
    ]
)
# decoder.summary()

### Training

In [7]:
# TODO: What is parametric_embedding?

n_epochs = 50
# n_epochs = 200

# Parametric embedding
# embedder = ParametricUMAP(n_epochs = n_epochs, parametric_embedding=True, verbose=True)

# No parametric_embedding
# embedder = ParametricUMAP(parametric_embedding=False, verbose=True)

## Encoder-decoder
embedder = ParametricUMAP(
    encoder=encoder,
    decoder=decoder,
    dims=dims,
    n_training_epochs=1,
    n_components=n_components,
    parametric_reconstruction=True,
    autoencoder_loss=True,
    reconstruction_validation=test_images,
    verbose=True,
)


embedding = embedder.fit_transform(train_images)

NameError: name 'ParametricUMAP' is not defined

### Plots

In [8]:
embedder._history.keys()

NameError: name 'embedder' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sc = ax.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=Y_train.astype(int),
    cmap="tab10",
    s=0.1,
    alpha=0.5,
    rasterized=True,
)
ax.axis("equal")
ax.set_title("UMAP in Tensorflow embedding", fontsize=20)
plt.colorbar(sc, ax=ax)

In [ ]:
fig, ax = plt.subplots()
ax.plot(embedder._history["loss"])
ax.set_ylabel("Cross Entropy")
ax.set_xlabel("Epoch")

#### Plot Encoder-Decoder Reconstruction

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(embedder._history["reconstruction_loss"], label="train")
ax.plot(embedder._history["val_reconstruction_loss"], label="valid")
ax.legend()
ax.set_ylabel("Cross Entropy")
ax.set_xlabel("Epoch")

In [ ]:
test_images_recon = embedder.inverse_transform(embedder.transform(test_images))

nex = 10
fig, axs = plt.subplots(ncols=10, nrows=2, figsize=(nex, 2))
for i in range(nex):
    axs[0, i].matshow(np.squeeze(test_images[i].reshape(28, 28, 1)), cmap=plt.cm.Greys)
    axs[1, i].matshow(
        tf.nn.sigmoid(np.squeeze(test_images_recon[i].reshape(28, 28, 1))),
        cmap=plt.cm.Greys,
    )
for ax in axs.flatten():
    ax.axis("off")

### Save/Load Model

In [ ]:
from umap.parametric_umap import load_ParametricUMAP

embedder.save("/tmp/model")
# embedder = load_ParametricUMAP('/tmp/model')